In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(InMemoryCache())
set_llm_cache(SQLiteCache("cache.db"))
set_debug(True)

chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

chat.predict("How do you make italian pasta")

Arrr matey! Me favorite food be a hearty stew made with fresh fish, vegetables, and a touch of rum to give it that extra kick! Aye, nothing beats a good meal after a long day of plunderin' the high seas! Arrr!

AIMessageChunk(content="Arrr matey! Me favorite food be a hearty stew made with fresh fish, vegetables, and a touch of rum to give it that extra kick! Aye, nothing beats a good meal after a long day of plunderin' the high seas! Arrr!")

In [ ]:
chat.predict("How do you make italian pasta")